# P1
### Arthur Bright
- project refined,

-- suggest a "minimum viable product", a baseline project

-- suggest how project will be evaluated (what methods you will use, what data you will use, etc.)  At this point you get to set the rules.  Again, you are establishing what you will do to "complete" the project.

-- provide any stretch goals.


- show current progress, any of the following:

--> project refinement

--> additional citations (paper, data, video links)

--> plots of data collected or generated (you can begin exploration in

        Audacity, Friture spectrum/waveform analyzer, or anything else)

        choosing data representative of what you are working with

--> algorithms run on any data, "working" or not.

--> use of web-based algorithms, collection of results


# Representation of Sung Notes

In [ ]:
# A class to represent a frequency, but can be instantiated using a piano note index as well.
#eg A0 = 0, A#0 = 1, etc
class Freq:
    def __init__(self, ind=None, freq=None):
        if(ind is not None):
            assert(isinstance(ind, int))
            assert(ind >= 0)
            # using A4 = 440, we have A0 = 440/16 = 27.5
            self.v = 27.5 * (2**(ind/12))
        elif(freq is not None):
            assert(isinstance(freq, float) or isinstance(freq, int))
            assert(freq > 0)
            self.v = float(freq)


# A class to represent a phoneme. Currently only supports a subset of japanese language
class Phoneme:
    consonants =   ["k", "g", "s", "z", "t", "d", "n", "h", "b", "p", "m", "r", "w", "j"]
    plosives =     ["p", "t", "b", "d", "g", "k"]
    fricatives =   ["s", "z", "h"]
    nasals =       ["m", "n"]
    approximants = ["w", "r", "j"]
    assert((set(plosives) | set(fricatives) | set(nasals) | set(approximants)) == set(consonants))

    vowels =       ["a", "o", "u", "i", "e"]

    def __init__(self, s):
        assert(s in self.consonants or s in self.vowels)
        self.v = s


# A class to represent a syllable. Captures a consonant + vowel OR lone vowel
class Syllable:
    def __init__(self, vowel: Phoneme, cons = None):
        assert(isinstance(vowel, Phoneme))
        assert(vowel.v in Phoneme.vowels)
        if (cons is not None):
            assert(isinstance(cons, Phoneme))
            assert(cons.v in Phoneme.consonants)

# A class to represent a sung note, which includes a (fundamental) frequency, syllable, and duration.
class Note:
    def __init__(self, f : Freq, s: Syllable, duration: float):
        assert(isinstance(f, Freq) and isinstance(s, Syllable) and isinstance(duration, float))
        self.f = f
        self.s = s

# A class to represent a period of silence.
class Silence:
    def __init__(self, duration: float):
        assert(isinstance(duration, float))
        self.duration = duration

# Interface to Convert Sung Notes to Signal

hi
